In [45]:
import pandas as pd
from funcoes import remover_missings,escolher_estrategia_imputacao,selecao_variaveis,perfil_base,aplicar_imputacao_treino,aplicar_imputacao_teste
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from typing import List, Tuple
import joblib
import pycaret
from typing import Dict, Tuple

In [46]:
train = pd.read_csv("train.csv",sep=",")
train.shape

(7315, 81)

In [47]:
#Remove colunas que possuem um percentual de valores ausentes (missings) maior ou igual ao valor definido em perc_miss.
#Definimos que variáveis com 50% de valores ausentes serão descartadas.
#Modelos de machine learning precisam de um conjunto de dados completo e representativo. 
#Se muitas variáveis tiverem altos níveis de missings, o modelo pode:
    #Perder generalização ao imputar valores incorretos. 
    #Diminuir a capacidade preditiva, pois pode aprender padrões errados.

treino_pos_miss = remover_missings(train, perc_miss= 50)

Colunas removidas(38): ['VAR_10', 'VAR_12', 'VAR_14', 'VAR_16', 'VAR_18', 'VAR_21', 'VAR_23', 'VAR_26', 'VAR_27', 'VAR_29', 'VAR_31', 'VAR_36', 'VAR_37', 'VAR_41', 'VAR_42', 'VAR_43', 'VAR_46', 'VAR_47', 'VAR_48', 'VAR_49', 'VAR_50', 'VAR_51', 'VAR_55', 'VAR_56', 'VAR_61', 'VAR_62', 'VAR_63', 'VAR_66', 'VAR_67', 'VAR_68', 'VAR_69', 'VAR_70', 'VAR_71', 'VAR_73', 'VAR_74', 'VAR_75', 'VAR_77', 'VAR_78']


In [48]:
#Estratégia 
#Média: Se a coluna for numérica, sem outliers e missing < 5%.
#Mediana: Se houver outliers e missing entre 5%-20%.
#Mediana: Para outros casos de dados numéricos. (ex: sem outlier e missing alto etc..)

#Utilizar modelos para imputação (criar um modelo que faça previsões dos valores faltantes) aumenta a complexidade, deploy e avaliação.
regra_imputacao = escolher_estrategia_imputacao(treino_pos_miss)

In [49]:
#Não vamos salvar as regras imputação pois esse modelo é apenas para auxiliar na seleção de variáveis
df_imputado, regra_imputacao, dict_mediana, dict_media = aplicar_imputacao_treino(treino_pos_miss,regra_imputacao)

In [50]:
#Selecionando vari

#classic (RFE - Recursive Feature Elimination)
#univariate (Testes estatísticos ANOVA/qui-quadrado)
#sequential (Sequential Feature Selection - SFS)
#Método de combinação das variáveis selecionadas.
#intersection: Mantém apenas as variáveis escolhidas por todos os métodos.
#union: Mantém todas as variáveis selecionadas por pelo menos um método.
#voting: Mantém variáveis selecionadas por pelo menos 2 dos métodos escolhidos.

variaveis_selecionadas = selecao_variaveis(
    data=df_imputado.drop(['id','safra'],axis=1), 
    target='y', 
    methods=['classic', 'univariate', 'sequential'], 
    selection_rule='union'
)
print(variaveis_selecionadas)

[LightGBM] [Info] Number of positive: 1446, number of negative: 3674
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000449 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6707
[LightGBM] [Info] Number of data points in the train set: 5120, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.282422 -> initscore=-0.932480
[LightGBM] [Info] Start training from score -0.932480
['VAR_15', 'VAR_9', 'VAR_54', 'VAR_3', 'VAR_5', 'VAR_1', 'VAR_28', 'VAR_60', 'VAR_45', 'VAR_6', 'VAR_58', 'VAR_38', 'VAR_32', 'VAR_19', 'VAR_8', 'VAR_17', 'VAR_33', 'VAR_30', 'VAR_2', 'VAR_11', 'VAR_24', 'VAR_25', 'VAR_65', 'VAR_59', 'VAR_53', 'VAR_20', 'VAR_57', 'VAR_44', 'VAR_72', 'VAR_40', 'VAR_39', 'VAR_34', 'VAR_52', 'VAR_13', 'VAR_35', 'VAR_76', 'VAR_7', 'VAR_4', 'VAR_64', 'VAR_22']


In [51]:
#Agora vamos retornar a base de train original e Salvar a base de train_selecionada (após seleção de variáveis)
train_selecionada = df_imputado[['id','safra','y']+variaveis_selecionadas]

In [52]:
train_selecionada.head()

,id,safra,y,VAR_15,VAR_9,VAR_54,VAR_3,VAR_5,VAR_1,VAR_28,...,VAR_39,VAR_34,VAR_52,VAR_13,VAR_35,VAR_76,VAR_7,VAR_4,VAR_64,VAR_22
0,7913,201405,0,80.41,700.00,700.00,0.0,73.96,0.0,0.0,...,695.0,80.96,261.72,180.82,594.09,1045.79,125.58,0.0,0,0.0
1,7343,201401,1,65.98,1000.00,1230.00,1.0,68.98,171.0,0.0,...,266.0,204.90,65.98,69.21,63.98,319.71,47.29,2.0,1,0.0
2,9046,201407,1,49.98,2209.00,2209.00,0.0,68.98,56.0,0.0,...,266.0,895.76,205.91,69.21,221.86,319.71,124.91,1.0,1,0.0
3,5227,201405,0,53.11,1000.00,1550.00,0.0,70.47,36.0,0.0,...,181.0,39.95,149.97,48.79,297.71,626.98,53.24,2.0,1,0.0
4,10491,201404,1,157.65,1479.39,1479.39,0.0,68.98,30.0,0.0,...,266.0,225.94,506.40,69.21,221.86,319.71,53.24,0.0,1,0.0


In [53]:
train_selecionada.shape

(7315, 43)

In [54]:
#Verificando nossa base após seleção de variáveis
resultado = perfil_base(train_selecionada, id_col='id', target_col='y', safra_col='safra')

Calcula métricas básicas do perfil da base de dados.
Shape da base: Essa base possui 7315 linhas e 43 colunas
Tipos de variáveis: {dtype('float64'): 37, dtype('int64'): 6}
IDs únicos: 7315
Taxa de maus (bad rate): Bons: 5249(71.8 %), Maus: 2066 (28.2%)
Volumetria das safras: {201401: 655, 201402: 714, 201403: 693, 201404: 768, 201405: 794, 201406: 730, 201407: 770, 201408: 737, 201409: 716, 201410: 738}




In [55]:
#Salvando a base de treino_selecionada
train_selecionada.to_csv("train_selecionada.csv",sep=",",index=False,header=True)